Prendiamo il file **Giocatori_Pred_Prob_Storiche.csv** e andiamo ad associare ad ogni giocatore il proprio score calcolato, prendendolo dal file **Score_Giocatori_Normalizzato.csv**

In [12]:
import pandas as pd

# Carica i file
df_pred_prob = pd.read_csv("Giocatori_Pred_Prob_Storiche.csv")
df_score = pd.read_csv("Score_Giocatori_Normalizzato.csv")
df_score.rename(columns={'Player': 'Giocatore'}, inplace=True)

# Estrai tutte le giornate presenti nel file delle predizioni
giornate_uniche = sorted(df_pred_prob['Giornata'].unique())

# Crea righe per ogni giocatore (con score) e ogni giornata
righe_completa = []
for _, row in df_score.iterrows():
    for giornata in giornate_uniche:
        righe_completa.append({
            'Giocatore': row['Giocatore'],
            'Giornata': giornata,
            'Team': row['Team'],
            'Role': row['Role'],
            'Score': row['Score']
        })

df_base = pd.DataFrame(righe_completa)

# Unisci con le predizioni (dove esistono)
df_unito = pd.merge(
    df_base,
    df_pred_prob[['Giocatore', 'Giornata', 'Prob_Gol', 'Pred_Gol_Pesata']],
    on=['Giocatore', 'Giornata'],
    how='left'
)

# Sostituisci i valori NaN con 0.0 (per portieri o assenti)
df_unito[['Prob_Gol', 'Pred_Gol_Pesata']] = df_unito[['Prob_Gol', 'Pred_Gol_Pesata']].fillna(0.0)

# Ordina ruoli se serve (opzionale)
ordine_ruoli = pd.CategoricalDtype(categories=['P', 'D', 'C', 'A'], ordered=True)
df_unito['Role'] = df_unito['Role'].astype(ordine_ruoli)

# Ordina per numero giornata (opzionale ma utile)
df_unito['Giornata_Num'] = df_unito['Giornata'].str.extract(r'G(\d+)').astype(int)
df_unito = df_unito.sort_values(by=['Role', 'Giornata_Num']).drop(columns='Giornata_Num')

# Salva il file finale
df_unito.to_csv("Giocatori_PredProbStoriche_Score_Completo.csv", index=False)
print("File salvato come Giocatori_PredProbStoriche_Score_Completo.csv")


File salvato come Giocatori_PredProbStoriche_Score_Completo.csv


Scarichiamo il file unito

In [13]:
from google.colab import files
files.download("Giocatori_PredProbStoriche_Score_Completo.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Adesso per ogni giornata aggiungiamo il numero di goal segnati nelle ultime 3 giornate

In [16]:
# Carica il file completo con score, predizione e probabilità
df_main = pd.read_csv("Giocatori_PredProbStoriche_Score_Completo.csv")

# Carica la matrice dei gol temporali
df_gol = pd.read_csv("GolGiocatoriTemporale.csv")

# Estrai le colonne giornate
giornate = [col for col in df_gol.columns if col.startswith("G") and col[1:].isdigit()]
df_gol.set_index("Giocatore", inplace=True)

# Funzione corretta per calcolare GolUlt3
def calcola_gol_ult3(giocatore, giornata_corrente):
    try:
        # Se la giornata non è nel dataset (es. G39), usa G38 come riferimento
        if giornata_corrente not in giornate:
            idx = len(giornate)  # G39 → index 38
        else:
            idx = giornate.index(giornata_corrente)
        giornate_prec = giornate[max(0, idx-3):idx]
        return df_gol.loc[giocatore, giornate_prec].sum()
    except:
        return 0

# Applica la funzione riga per riga
df_main["GolUlt3"] = df_main.apply(
    lambda row: calcola_gol_ult3(row["Giocatore"], row["Giornata"]),
    axis=1
)

# Salva il nuovo file aggiornato
df_main.to_csv("Giocatori_PredProbStoriche_Score_GolUlt3.csv", index=False)
print("File salvato come Giocatori_PredProbStoriche_Score_GolUlt3.csv")


File salvato come Giocatori_PredProbStoriche_Score_GolUlt3.csv


In [17]:
files.download("Giocatori_PredProbStoriche_Score_GolUlt3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Gestione etichette

Obiettivo

Calcolare per ogni squadra di Serie A il numero totale di gol subiti nelle ultime 3 giornate, per ciascuna giornata dalla G4 alla G39.

* Carica e normalizza il file Calendario2324.csv
* Estrae il numero di gol subiti da ciascuna squadra in ogni giornata
* Scorre ogni squadra e calcola la somma dei gol subiti nelle 3 giornate precedenti, a partire dalla G4

In [19]:
from collections import defaultdict

# Carica il file calendario
df = pd.read_csv("Calendario2324.csv")

# Normalizza i nomi squadra
mappa_squadre = {
    "Hellas Verona": "Verona",
    "Verona": "Verona"
}
df["Casa"] = df["Casa"].replace(mappa_squadre)
df["Ospite"] = df["Ospite"].replace(mappa_squadre)

# Assicura che la colonna Giornata sia intera
df['Giornata'] = df['Giornata'].astype(int)

# Funzione per estrarre gol casa e trasferta da una stringa tipo "2-1"
def estrai_gol(risultato):
    try:
        gol_casa, gol_trasferta = map(int, risultato.split('-'))
        return gol_casa, gol_trasferta
    except:
        return None, None

# Applica la funzione
df[['Gol_Casa', 'Gol_Trasferta']] = df['Risultato'].apply(lambda x: pd.Series(estrai_gol(x)))

# Dizionario per registrare i gol subiti da ogni squadra per ogni giornata
gol_subiti = defaultdict(lambda: defaultdict(int))

# Registra i gol subiti per ogni squadra
for _, row in df.iterrows():
    giornata = row['Giornata']
    casa = row['Casa']
    ospite = row['Ospite']
    gol_casa = row['Gol_Casa']
    gol_trasferta = row['Gol_Trasferta']

    if pd.notna(gol_casa) and pd.notna(gol_trasferta):
        gol_subiti[casa][giornata] += gol_trasferta
        gol_subiti[ospite][giornata] += gol_casa

# Calcola i gol subiti nelle ultime 3 giornate per ogni squadra dalla giornata 4 in poi
rows = []
giornata_max = max(39, df['Giornata'].max())  # Forza almeno G39

for squadra in gol_subiti:
    for giornata in range(4, giornata_max + 1):
        somma_subiti = sum(gol_subiti[squadra].get(g, 0) for g in range(giornata - 3, giornata))
        rows.append({
            'Squadra': squadra,
            'Giornata': giornata,
            'Gol_Subiti_Ultime3': somma_subiti
        })

# Crea il DataFrame finale
df_subiti = pd.DataFrame(rows)

# Salva il risultato con nome coerente
output_file = "Gol_Subiti_Ultime3_G4_G39.csv"
df_subiti.to_csv(output_file, index=False)
print(f"File salvato come {output_file}")

try:
    files.download(output_file)
except:
    pass


File salvato come Gol_Subiti_Ultime3_G4_G39.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Obiettivo

Integrare nel dataset principale il numero di gol subiti nelle ultime 3 giornate per i portieri, sostituendo il valore generico GolUlt3 con quello difensivo calcolato per squadra.

* Carica:

  * il file con score, probabilità e forma (Giocatori_PredProbStoriche_Score_GolUlt3.csv)
  * il file con i gol subiti per squadra (Gol_Subiti_Ultime3_G4_G39.csv)
* Identifica i portieri e, per ciascuno, sostituisce GolUlt3 con i gol subiti dalla propria squadra nelle ultime 3 giornate
* Mantiene invariato il campo GolUlt3 per tutti gli altri ruoli

In [20]:
# Carica i due file
df = pd.read_csv("Giocatori_PredProbStoriche_Score_GolUlt3.csv")
df_subiti = pd.read_csv("Gol_Subiti_Ultime3_G4_G39.csv")



# Estrai il numero della giornata come intero (es. G4 → 4)
df["Giornata_Num"] = df["Giornata"].str.extract(r'G(\d+)').astype(int)

# Crea una copia della colonna GolUlt3 per sicurezza
df["GolUlt3_originale"] = df["GolUlt3"]

# Sostituisci GolUlt3 solo per i portieri
def sostituisci_se_portiere(row):
    if row["Role"] == "P":
        record = df_subiti[
            (df_subiti["Squadra"] == row["Team"]) &
            (df_subiti["Giornata"] == row["Giornata_Num"])
        ]
        if not record.empty:
            return int(record["Gol_Subiti_Ultime3"].values[0])
    return row["GolUlt3"]

df["GolUlt3"] = df.apply(sostituisci_se_portiere, axis=1)

# Rimuovi colonna temporanea
df.drop(columns=["Giornata_Num", "GolUlt3_originale"], inplace=True)

# Salva il file mantenendo la stessa struttura
df.to_csv("Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv", index=False)
print("File salvato come Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv")

files.download("Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv")


File salvato come Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Obiettivo

Calcolare uno score composito per ogni giocatore e assegnare un’etichetta di consigliabilità (consigliato, non consigliato, sorpresa) per ciascuna giornata da G4 a G38.

* Carica il file completo con tutte le feature (Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv)
* Esegue la normalizzazione di Score (su scala 0–10) e GolUlt3 (su scala 0–3)

* Calcola lo score composito:
  * Portieri: 0.7 × score + 0.3 × gol subiti
* Giocatori di movimento: combinazione di probabilità, predizione, score e forma recente

* Assegna l’etichetta:

  * 1 = consigliato
  * 2 = sorpresa
  * 0 = non consigliato
  * (non viene assegnata alcuna etichetta alla G39)

In [21]:
# Carica il file con score, probabilità, predizione e GolUlt3
df = pd.read_csv("Giocatori_PredProbStoriche_Score_GolUlt3_Con_Portieri.csv")

# Normalizzazione
df["Score_norm"] = df["Score"] / 10.0  # score massimo atteso = 10
df["GolUlt3_norm"] = df["GolUlt3"] / 3.0  # massimo 3 gol in 3 giornate

# Salva il file intermedio con le normalizzazioni
df.to_csv("Giocatori_Normalizzati.csv", index=False)
print("File intermedio salvato come Giocatori_Normalizzati.csv")
files.download("Giocatori_Normalizzati.csv")


# Calcolo score composito differenziato
def calcola_score_composito(row):
    if row["Role"] == "P":
        return 0.7 * row["Score_norm"] + 0.3 * row["GolUlt3_norm"]
    else:
        return (
            0.3 * row["Prob_Gol"] +
            0.3 * row["Pred_Gol_Pesata"] +
            0.5 * row["Score_norm"] +
            0.2 * row["GolUlt3_norm"]
        )

df["Score_Composito"] = df.apply(calcola_score_composito, axis=1)

def assegna_etichetta(row):
    if row["Role"] == "P":
        if row["Score_norm"] == 0:
            return 0  # portiere mai utilizzato → non consigliato
        elif row["Score_Composito"] >= 0.50:
            return 1  # consigliato
        elif row["GolUlt3_norm"] < 1:
            return 2  # sorpresa
        else:
            return 0  # non consigliato
    else:
        if row["GolUlt3_norm"] > 0:
            if row["Score_Composito"] >= 0.40:
                return 1  # consigliato
            elif row["Score_Composito"] >= 0.15:
                return 2  # sorpresa
            else:
                return 0  # non consigliato
        else:
            if row["Score_Composito"] >= 0.45:
                return 1  # consigliato anche senza gol recenti
            else:
                return 0

# Non assegnare etichetta alla G39
df["Etichetta"] = df.apply(
    lambda row: assegna_etichetta(row) if int(row["Giornata"].replace("G", "")) < 39 else None,
    axis=1
)

# Salva il file finale
df.to_csv("Giocatori_Con_Etichetta.csv", index=False)
print("File salvato come Giocatori_Con_Etichetta.csv")

files.download("Giocatori_Con_Etichetta.csv")


File intermedio salvato come Giocatori_Normalizzati.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File salvato come Giocatori_Con_Etichetta.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Per vedere meglio i risultati sono stati filtrati e ordinati i giocatori della giornata G38 con etichetta assegnata (0, 1, 2), salvando il file finale Consigliati_G38.csv.

In [22]:
# Carica il file etichettato
df = pd.read_csv("Giocatori_Con_Etichetta.csv")

# Filtra solo i consigliati (Etichetta = 1) per la giornata G38
df_g38 = df[(df["Giornata"] == "G38") & ((df["Etichetta"] == 1) | (df["Etichetta"] == 2) | (df["Etichetta"] == 0)) ].copy()

# Ordine ruoli
ordine_ruoli = pd.CategoricalDtype(categories=["P", "D", "C", "A"], ordered=True)
df_g38["Role"] = df_g38["Role"].astype(ordine_ruoli)

# Ordina per ruolo e poi per squadra
df_g38 = df_g38.sort_values(by=["Role", "Team", "Giocatore"])

# Mostra i risultati
from IPython.display import display
display(df_g38)

# salva il file dei consigliati della G38
df_g38.to_csv("Consigliati_G38.csv", index=False)
print("File salvato come Consigliati_G38.csv")

files.download("Consigliati_G38.csv")


Giocatore Giornata      Team Role     Score  Prob_Gol  \
2654   andrea bartoccioni      G38  AC Milan    P  0.000000    0.0000   
2649      antonio mirante      G38  AC Milan    P  0.000000    0.0000   
2657            lapo nava      G38  AC Milan    P  0.000000    0.0000   
2659     lorenzo torriani      G38  AC Milan    P  0.000000    0.0000   
2612     marco sportiello      G38  AC Milan    P  0.539302    0.0000   
...                   ...      ...       ...  ...       ...       ...   
24615    siren diao balde      G38    Verona    A  0.001946    0.0000   
24583     stefan mitrovic      G38    Verona    A  0.123591    0.0000   
24553        thomas henry      G38    Verona    A  0.692889    0.0616   
24546      tijjani noslin      G38    Verona    A  1.780393    0.1104   
24598        yayah kallon      G38    Verona    A  0.001946    0.0000   

       Pred_Gol_Pesata  GolUlt3  Score_norm  GolUlt3_norm  Score_Composito  \
2654            0.0000        0    0.000000      0.000000         0.000000   
2649            0.0000        0    0.000000      0.000000         0.000000   
2657            0.0000        0    0.000000      0.000000         0.000000   
2659            0.0000        0    0.000000      0.000000         0.000000   
2612            0.0000        0    0.053930      0.000000         0.037751   
...                ...      ...         ...           ...              ...   
24615           0.0000        0    0.000195      0.000000         0.000097   
24583           0.0000        0    0.012359      0.000000         0.006180   
24553           0.0861        0    0.069289      0.000000         0.078954   
24546           0.1278        1    0.178039      0.333333         0.227146   
24598           0.0000        0    0.000195      0.000000         0.000097   

       Etichetta  
2654         0.0  
2649         0.0  
2657         0.0  
2659         0.0  
2612         2.0  
...          ...  
24615        0.0  
24583        0.0  
24553        0.0  
24546        2.0  
24598        0.0  

[689 rows x 12 columns]

File salvato come Consigliati_G38.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Preparazione del dataset per la fase di classificazione supervisionata



In [23]:
# 1. Carica il file completo
df = pd.read_csv("Giocatori_Con_Etichetta.csv")

# 2. Seleziona solo le colonne desiderate per la fase di classificazione
colonne_finali = ['Giocatore', 'Giornata', 'Team', 'Role', 'Score_norm', 'Prob_Gol', 'Pred_Gol_Pesata', 'GolUlt3', 'Etichetta']
df_finale = df[colonne_finali].copy()

# 3. Controllo rapido su valori nulli
print("Valori nulli per colonna:")
print(df_finale.isnull().sum())

# 4. Salva il nuovo CSV pronto per la Fase di Classificazione supervisionata
df_finale.to_csv("Dataset_Per_Classificazione.csv", index=False)
print("Dataset salvato come 'Dataset_Per_Classificazione.csv'")

files.download("Dataset_Per_Classificazione.csv")


Valori nulli per colonna:
Giocatore            0
Giornata             0
Team                 0
Role                 0
Score_norm           0
Prob_Gol             0
Pred_Gol_Pesata      0
GolUlt3              0
Etichetta          689
dtype: int64
Dataset salvato come 'Dataset_Per_Classificazione.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>